In [121]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.nn as nn

In [122]:
batch_size = 4
TRAIN_SIZE=1e4
epochs=3
iters_per_epoch=int(TRAIN_SIZE//batch_size)

In [123]:
# 准备minist数据集
data_dir = "data"
transform = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,)),  # Mean and std of MNIST
    ]
)

In [124]:
train_dataset = datasets.MNIST(
    root=data_dir, train=True, transform=transform, download=True
)
test_dataset = datasets.MNIST(
    root=data_dir, train=False, transform=transform, download=True
)

In [125]:
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

In [126]:
class MLP(nn.Module):
    def __init__(self, in_features, hidden_features, num_classes):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(in_features, hidden_features)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_features, num_classes)

    def forward(self, x: torch.Tensor):
        x = x.reshape(batch_size, 28 * 28)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

In [127]:
model=MLP(in_features=784,hidden_features=256,num_classes=10).cuda()

In [128]:
def train(
    model: nn.Module,
    criterion: nn.functional,
    optimizer: torch.optim.Optimizer,
    epoch: int,
):
    model.train()
    running_loss: float = 0
    # for i in range(iters_per_epoch):
    for i, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        outputs = model(data.cuda())
        loss = criterion(outputs, target.cuda())
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        running_loss += loss.item()
        if i % 100 == 99 or i == 0:
            print(f"Iteration {i}, Loss {running_loss/(i+1)}")

In [129]:
criterion=nn.CrossEntropyLoss()
optimizer=torch.optim.SGD(model.parameters(),lr=0.001)

In [130]:
train(model,criterion,optimizer,30)

Iteration 0, Loss 2.370542049407959
Iteration 99, Loss 2.238724137544632
Iteration 199, Loss 2.160097403526306
Iteration 299, Loss 2.0759207757314044
Iteration 399, Loss 1.9999788796901703
Iteration 499, Loss 1.9289768240451812
Iteration 599, Loss 1.8484020765622458
Iteration 699, Loss 1.7785841651473726
Iteration 799, Loss 1.7061303236335517
Iteration 899, Loss 1.6388714031378429
Iteration 999, Loss 1.5775710051059724
Iteration 1099, Loss 1.5260332587361336
Iteration 1199, Loss 1.4731472800920407
Iteration 1299, Loss 1.4239216938385597
Iteration 1399, Loss 1.3844142674122537
Iteration 1499, Loss 1.3450427582859994
Iteration 1599, Loss 1.3101327836792915
Iteration 1699, Loss 1.2784056954261134
Iteration 1799, Loss 1.246722096958094
Iteration 1899, Loss 1.2145366725953002
Iteration 1999, Loss 1.1866362716853618
Iteration 2099, Loss 1.1573888855037235
Iteration 2199, Loss 1.129998024356636
Iteration 2299, Loss 1.1080167867534834
Iteration 2399, Loss 1.0873312521260232
Iteration 2499, Los

In [131]:
def evaluate(model:nn.Module,test_data_loader:DataLoader):
    model.eval()
    correct=0
    total=0
    with torch.no_grad():
        for data in test_data_loader:
            images,labels=data
            outputs=model(images.cuda())
            _,predicted=torch.max(outputs.data,1)
            total+=labels.size(0)
            correct+=(predicted==labels.cuda()).sum().item()
    print(f"Accuracy: {100*correct/total}")

In [132]:
evaluate(model=model,test_data_loader=test_loader)

Accuracy: 91.74
